### 安装

In [2]:
!bash -c 'cd .. && ./scripts/install_miqroforge.sh'

docker is already installed
python is already installed
NFS Server is running
k3s is already installed
[+] Running 3/3
 ✔ Container miqroforge-mysql  Running                                     0.0s 
 ✔ Container miqroforge-minio  Running                                     0.0s 
 ✔ Container miqroforge-web    Running                                     0.0s 
Miqroforge started successfully
You can access Miqroforge at http://172.26.232.156:30080/miqroforge-frontend/
miqroforge is already installed
usage: miqroforge [-h] <command> ...

MiqroForge 命令行工具

positional arguments:
  <command>
    task      查看计算任务
    node      查看节点模板信息

options:
  -h, --help  show this help message and exit


### 测试连接状态

In [1]:
#!/usr/bin/env python3

from miqroforge.managers import ServiceManager
import logging

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def test_service_connection():
    """测试服务连接"""
    print("=== 测试服务连接 ===\n")
    
    manager = ServiceManager()
    
    # 测试Kubernetes连接
    print("测试Kubernetes连接...")
    if manager.init_kubernetes():
        if manager.k8s and manager.k8s.test_connection():
            print("  ✓ Kubernetes连接测试成功")
        else:
            print("  ✗ Kubernetes连接测试失败")
    else:
        print("  ✗ Kubernetes初始化失败")
    
    # 测试Docker连接
    print("\n测试Docker连接...")
    if manager.init_docker():
        if manager.docker and manager.docker.test_connection():
            print("  ✓ Docker连接测试成功")
        else:
            print("  ✗ Docker连接测试失败")
    else:
        print("  ✗ Docker初始化失败")
    
    # 测试MySQL连接
    print("\n测试MySQL连接...")
    if manager.init_mysql():
        if manager.mysql and manager.mysql.is_connected():
            print("  ✓ MySQL连接测试成功")
        else:
            print("  ✗ MySQL连接测试失败")
    else:
        print("  ✗ MySQL初始化失败")
    
    # 断开所有连接
    manager.disconnect_all()

def test_all_services_init():
    """测试一键初始化所有服务"""
    print("\n=== 测试一键初始化所有服务 ===\n")
    
    manager = ServiceManager()
    
    if manager.init_all_services():
        print("✓ 所有服务初始化成功")
    else:
        print("⚠ 部分服务初始化失败")
    
    # 显示服务状态
    print("\n服务状态:")
    status = manager.get_service_status()
    for service, state in status.items():
        print(f"  {service}: {state}")
    
    # 断开所有连接
    manager.disconnect_all()

if __name__ == "__main__":
    test_service_connection()
    test_all_services_init()


INFO: Kubernetes客户端初始化成功 (config: /root/.kube/config, context: default)
INFO: Kubernetes客户端初始化成功
INFO: Docker客户端连接成功 (host: unix:///var/run/docker.sock, tls_verify: False)
INFO: Docker客户端初始化成功
INFO: MySQL连接成功
INFO: MySQL客户端初始化成功
INFO: MySQL连接已断开
INFO: 所有服务连接已断开
INFO: Kubernetes客户端初始化成功 (config: /root/.kube/config, context: default)
INFO: Kubernetes客户端初始化成功
INFO: Docker客户端连接成功 (host: unix:///var/run/docker.sock, tls_verify: False)
INFO: Docker客户端初始化成功
INFO: MySQL连接成功
INFO: MySQL客户端初始化成功


=== 测试服务连接 ===

测试Kubernetes连接...
  ✓ Kubernetes连接测试成功

测试Docker连接...
  ✓ Docker连接测试成功

测试MySQL连接...
  ✓ MySQL连接测试成功

=== 测试一键初始化所有服务 ===



INFO: SQLAlchemy连接成功
INFO: SQLAlchemy客户端初始化成功
INFO: 服务初始化完成: 4/4 个服务连接成功
INFO: MySQL连接已断开
INFO: 所有服务连接已断开


✓ 所有服务初始化成功

服务状态:
  kubernetes: 已连接
  docker: 已连接
  mysql: 已连接
  sqlalchemy: 已连接


### 查询节点信息

In [2]:
!miqroforge node

节点 1
--------------------------------------------------
id:                  684bee08-a78d-4b1f-87a8-91910ca81f38
type:                I
name:                {"cn":"加载xyz","en":"Load Xyz"}
description:         -
version:             1.0.0
color:               #8e294a
tag:                 Build Struct
input:               {"web":[{"var":"xyz_file_path","name":{"cn":"分子文件","en":"molecule_files"},"descr
                     iption":"","ui":{"mpi_files":""}}]}
output:              {"downstream":[{"var":"xyz_file_path","name":{"cn":"Structure","en":"Structure"}
                     ,"description":"","default_value":"${xyz_file_path}.xyz"}],"web":[]}
performance_config_path: 
example_config_path: 
contact:             {"name":"","email":""}
execution_command:   
image:               NULL
节点 2
--------------------------------------------------
id:                  b2e1e7e2-2b2a-4e2a-8e2a-2b2a4e2a8e2a
type:                D
name:                {"cn":"分子预览","en":"Molecule Preview"}
description

### 查询任务列表

In [3]:
!miqroforge task


+----------+------------------+---------+---------------------+---------------------+---------------------+-------------------------------+
| 任务ID   | 任务名称         | 状态    | 开始时间            | 结束时间            | 创建时间            | 错误信息                      |
+==========+==================+=========+=====================+=====================+=====================+===============================+
| 265      | kVjau51tbtsTCo3W | SUCCEED | 2025-08-12 10:58:43 | 2025-08-12 11:25:25 | 2025-08-12 10:58:43 | -                             |
+----------+------------------+---------+---------------------+---------------------+---------------------+-------------------------------+
| 264      | KQrOUGLyawr1zB1c | FAILED  | 2025-08-12 09:56:50 | 2025-08-12 09:57:05 | 2025-08-12 09:56:50 | Starting calculation with...  |
+----------+------------------+---------+---------------------+---------------------+---------------------+-------------------------------+
| 263      | TfES5FvyXS3ReKTl | FAILED  | 2

### 查询任务节点列表

In [4]:
!miqroforge task --id 265


任务ID 265 的节点列表:

+----------+----------+------------+------------+---------+--------------------+------------+---------------------+
| 节点ID   | 任务ID   | 中文名称   | 英文名称   | 状态    | 数据目录           | 作业数量   | 创建时间            |
+==========+==========+============+============+=========+====================+============+=====================+
| 1597     | 265      |            |            | SUCCEED |                    |            | 2025-08-12 10:58:43 |
+----------+----------+------------+------------+---------+--------------------+------------+---------------------+
| 1598     | 265      |            |            | SUCCEED |                    |            | 2025-08-12 10:58:43 |
+----------+----------+------------+------------+---------+--------------------+------------+---------------------+
| 1599     | 265      |            |            | SUCCEED | task/265/node-1599 | 1          | 2025-08-12 10:58:43 |
+----------+----------+------------+------------+---------+--------------------+

### 查询任务节点参数列表

In [6]:
!miqroforge task --id 265 --node-id 1600


节点ID 1600 的参数列表:

+----------+----------+----------+------------+------------+--------------+--------------------+---------------------------------------------------+
| 参数ID   | 任务ID   | 节点ID   | 参数类型   | 中文名称   | 英文名称     | 参数代码           | 参数值                                            |
+==========+==========+==========+============+============+==============+====================+===================================================+
| 26619    | 265      | 1600     | 输入参数   | 活跃电子数 | active_elec  | activate_electrons | [3,3]                                             |
+----------+----------+----------+------------+------------+--------------+--------------------+---------------------------------------------------+
| 26620    | 265      | 1600     | 输入参数   | 活化轨道   | active_orbit | activate_orbit     | [3,4,5,6,7,8,9,10,11]                             |
+----------+----------+----------+------------+------------+--------------+--------------------+---------------------------------

### 添加节点

In [ ]:
!miqroforge node --add miqroera01/mf-pyscf-hf:v1.1 /app/Pyscf-HF

adding node:
  image: miqroera01/miqroforge_draw_image:v1.0
  app_path: /app/draw_image
Image miqroera01/miqroforge_draw_image:v1.0 already exists, skip pulling.
get node description failed: Command 'bash -c 'cd /app/draw_image && cat help.md'' in image 'miqroera01/miqroforge_draw_image:v1.0' returned non-zero exit status 1: b'cat: help.md: No such file or directory\n'
Starting to process node, ID: 684bee08-a78d-4b1f-87a8-01010ca81f38
Checking if node already exists...
Node 684bee08-a78d-4b1f-87a8-01010ca81f38 already exists, performing update operation...
Node updated successfully, ID: 684bee08-a78d-4b1f-87a8-01010ca81f38
Node processing completed, ID: 684bee08-a78d-4b1f-87a8-01010ca81f38
Importing image to k3s: miqroera01/miqroforge_draw_image:v1.0
miqroera01/miqroforge_draw_image:v1.0: resolving      |--------------------------------------| 
elapsed: 0.1 s                         total:   0.0 B (0.0 B/s)                                         




miqroera01/miqroforge_draw_image:v

In [ ]:
from uuid import uuid4


print(uuid4())

028e93ee-ca80-4239-a0f2-6bf673eb9fe7
